In [1]:
!pip install ultralytics --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.2 MB/s eta 0:00:00


In [2]:
!ls -l ../input/road-damage-dataset-potholes-cracks-and-manholes/data

total 0
drwxr-xr-x 2 nobody nogroup 0 Dec 16 18:53 images
drwxr-xr-x 2 nobody nogroup 0 Dec 16 18:53 labels


In [3]:
import os
import shutil
import yaml
import gc
from ultralytics import YOLO, settings
import torch
import numpy as np

# Riproducibilità
torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Impostazioni generali
PROJECT = "pothole-detector-NatureSR"
RANDOM_SEED = 0
IOU_THRESHOLD = 0.7
CONF_THRESHOLD = 0.25
INPUT_SIZE = 640
PATIENCE = 10
EPOCHS = 100

# Dataset paths
data_dir = "/kaggle/input/road-damage-dataset-potholes-cracks-and-manholes/data"

# Configurazione parametri (run gj4071db)
config = {
    'batch': 8,
    'optimizer': "RAdam",
    'lr0': 0.007023087386876883,
    'lrf': 0.02559244954708425,
    'weight_decay': 0.005034138135400871,
    'momentum': 0.09079056013311568,
    'dropout': 0.11321865578015432
}

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
def create_yaml():
    """Crea il file YAML che punta direttamente alla tua struttura"""
    yaml_path = "data.yaml"
    
    data = {
        'path': data_dir,
        'train': 'images',
        'val': 'images',  # Usa lo stesso set per train e val
        'nc': 3,
        'names': ['pothole', 'crack', 'manhole']
    }
    
    with open(yaml_path, 'w') as f:
        yaml.dump(data, f)
    
    print(f"✅ File YAML creato: {yaml_path}")
    return yaml_path

In [5]:
def train_model(data_yaml_path, config):
    """Training del modello YOLO"""
    
    print(f"\n🚀 Avvio training con configurazione:")
    for key, value in config.items():
        print(f"   - {key}: {value}")
    
    # Inizializza modello
    model = YOLO("yolo11n.pt")
    
    # Training
    print("\n⏳ Training in corso...")
    results = model.train(
        data=data_yaml_path,
        epochs=EPOCHS,
        batch=config['batch'],
        imgsz=INPUT_SIZE,
        optimizer=config['optimizer'],
        lr0=config['lr0'],
        lrf=config['lrf'],
        weight_decay=config['weight_decay'],
        momentum=config['momentum'],
        dropout=config['dropout'],
        patience=PATIENCE,
        device=0,
        amp=True,
        seed=RANDOM_SEED,
        deterministic=True,
        project=PROJECT,
        name="training",
        exist_ok=True,
        val=True,
        save=True,
        plots=True,
        verbose=True,
        workers=4,
    )
    
    # Validation
    print("\n📊 Validazione finale...")
    val_results = model.val(
        data=data_yaml_path,
        iou=IOU_THRESHOLD,
        conf=CONF_THRESHOLD
    )
    
    metrics = val_results.box
    class_names = ['pothole', 'crack', 'manhole']
    
    # Stampa risultati per classe
    print("\n📈 Risultati per classe:")
    print("-" * 70)
    print(f"{'Classe':<12} {'Precision':<12} {'Recall':<12} {'F1':<12} {'mAP50':<12}")
    print("-" * 70)
    
    for i, name in enumerate(class_names):
        precision = float(metrics.p[i])
        recall = float(metrics.r[i])
        f1 = float(metrics.f1[i])
        ap50 = float(metrics.ap50[i])
        
        print(f"{name:<12} {precision:<12.4f} {recall:<12.4f} {f1:<12.4f} {ap50:<12.4f}")
    
    # Stampa medie
    print("-" * 70)
    print(f"{'MEDIA':<12} {float(metrics.mp):<12.4f} {float(metrics.mr):<12.4f} "
          f"{float(metrics.f1.mean()):<12.4f} {float(metrics.map50):<12.4f}")
    print("-" * 70)
    
    print(f"\n✅ Training completato!")
    print(f"   - mAP50: {float(metrics.map50):.4f}")
    print(f"   - mAP50-95: {float(metrics.map):.4f}")
    print(f"   - Fitness: {float(metrics.fitness()):.4f}")

    #Exporting ONNX
    model.export(format="onnx")
    
    # Cleanup
    del model
    torch.cuda.empty_cache()
    gc.collect()
    
    return val_results

In [6]:
# Crea YAML
data_yaml_path = create_yaml()

# Training
results = train_model(data_yaml_path, config)

print("\n🏁 Processo completato!")

✅ File YAML creato: data.yaml

🚀 Avvio training con configurazione:
   - batch: 8
   - optimizer: RAdam
   - lr0: 0.007023087386876883
   - lrf: 0.02559244954708425
   - weight_decay: 0.005034138135400871
   - momentum: 0.09079056013311568
   - dropout: 0.11321865578015432

⏳ Training in corso...
Ultralytics 8.3.239 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.11321865578015432, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, ko

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       2009       4737      0.579       0.54      0.551      0.267
               pothole        795       1261      0.537      0.452      0.454      0.199
                 crack       1375       2519      0.522      0.357      0.392      0.168
               manhole        759        957      0.678       0.81      0.808      0.433
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /kaggle/working/pothole-detector-NatureSR/training

📊 Validazione finale...
Ultralytics 8.3.239 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11n summary (fused): 100 layers, 2,582,737 parameters, 0 gradients
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 144.7±76.1 MB/s, size: 85.8 KB)
val: Scanning /kaggle/input/road-damage-dataset-potholes-cracks-and-manholes/data/labels... 2009 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 2009/2009 833.4it/s 2.4s
WARNING ⚠️ val: Cache directory /kaggle/input/r

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all       2009       4737      0.649      0.484      0.572      0.311
               pothole        795       1261      0.611      0.392      0.474      0.244
                 crack       1375       2519      0.593      0.275       0.43      0.219
               manhole        759        957      0.744      0.785      0.812       0.47
Speed: 0.5ms preprocess, 1.5ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /kaggle/working/runs/detect/val

📈 Risultati per classe:
----------------------------------------------------------------------
Classe       Precision    Recall       F1           mAP50       
----------------------------------------------------------------------
pothole      0.6106       0.3918       0.4773       0.4744      
crack        0.5928       0.2751       0.3758       0.4297      
manhole      0.7436       0.7847       0.7636       0.8118      
----------------------------------------------------------------------
MEDIA        0.